In [66]:
import cv2 #opencv
import mediapipe as mp
import math
import csv
import datetime
import pandas as pd
import os
import numpy as np
from mediapipe_estimate import estimate
pd.options.mode.chained_assignment = None 

In [67]:
directory_path = 'result_statistics/'
original_videos_path='cropped_videos/'

In [68]:
test_dat=data=pd.read_csv('golfDB_front_orig.csv',index_col='id')
test_dat=test_dat[['youtube_id','events_2','events_5','events_7']]
test_dat['events_5']=test_dat['events_5']-test_dat['events_2']+15
test_dat['events_7']=test_dat['events_7']-test_dat['events_2']+15

test_dat=test_dat.drop(['events_2'],axis=1)
test_dat=test_dat.drop_duplicates(['youtube_id'],keep='last')
test_dat

,youtube_id,events_5,events_7
id,,,
8,-M5SITXMA2Y,45,53
10,gOBVh7fzyZo,39,47
14,vN3Uc_EhnnY,41,49
19,iW323nsTGtU,49,57
21,xD6KDqPF9cc,39,45
...,...,...,...
1370,g90r9cs1tTw,37,43
1372,eNlBbMI-MhQ,49,59
1374,6K1FD0sOwoI,54,63


In [69]:
def evaluate_correctness(data, swing_part,swing_part_id,address_frame=0):
    midpoint_x=data['midpoint_x'].iloc[swing_part_id]
    right_wrist_x=data['left_wrist_x'].iloc[swing_part_id]
    arm_angle=data['arm_angle'].iloc[swing_part_id]
    pelvis_angle=data['pelvis_angle'].iloc[swing_part_id]
    left_shoulder_x=data['left_shoulder_x'].iloc[swing_part_id]
    left_ankle_x=data['left_ankle_x'].iloc[swing_part_id]
    knee_angle=data['knee_angle'].loc[swing_part_id]

    
    if swing_part=='address':
        correct_midpoint=0
        correct_arm_angle=0
        if midpoint_x-right_wrist_x<0:
            correct_midpoint=1
        if 165<=arm_angle<=180:
            correct_arm_angle=1
        return correct_midpoint,correct_arm_angle
    
    elif swing_part=='top':
        head_point_current=np.array([data['nose_y'].iloc[swing_part_id],data['nose_x'].iloc[swing_part_id]])
        head_point_address=np.array([data['nose_y'].iloc[address_frame],data['nose_x'].iloc[address_frame]])
        correct_pelvis=0
        correct_head=0
        correct_arm_angle=0
        if 165<=pelvis_angle<=180:
            correct_pelvis=1
        if 130<=arm_angle<=150:
            correct_arm_angle=1
        if np.array(head_point_address).shape==(2,1):
            head_point_address=head_point_address.flatten()
            head_point_current=head_point_current.flatten()
        if np.linalg.norm(head_point_current-head_point_address)<=20:
            correct_head=1
        return correct_pelvis,correct_head,correct_arm_angle

    elif swing_part=='contact':
        head_point_current=np.array([data['nose_y'].iloc[swing_part_id],data['nose_x'].iloc[swing_part_id]])
        head_point_address=np.array([data['nose_y'].iloc[address_frame],data['nose_x'].iloc[address_frame]])
        correct_shoulder_ankle=0
        correct_head=0
        correct_knee_angle=0
        correct_arm_angle=0
        if np.array(head_point_address).shape==(2,1):
            head_point_address=head_point_address.flatten()
            head_point_current=head_point_current.flatten()
        if np.linalg.norm(head_point_current-head_point_address)<=20:
            correct_head=1
        if left_ankle_x-left_shoulder_x<0:
            correct_shoulder_ankle=1
        if 165<=knee_angle<=180:
            correct_knee_angle=1
        if 165<=arm_angle<=180:
            correct_arm_angle=1
        return correct_head,correct_shoulder_ankle,correct_knee_angle,correct_arm_angle

        
         
            

In [70]:
def plot_points(data,frame,swing_part,swing_part_id,address_frame=None):
    if swing_part=='address':
        correct_midpoint,correct_arm_angle=evaluate_correctness(data, swing_part,swing_part_id)
        cv2.circle(frame, (data['midpoint_x'].iloc[swing_part_id], data['midpoint_y'].iloc[swing_part_id]), 6, (0, 255, 0), -1)
        cv2.line(frame, (data['right_ankle_x'].iloc[swing_part_id], data['right_ankle_y'].iloc[swing_part_id]), (data['left_ankle_x'].iloc[swing_part_id], data['left_ankle_y'].iloc[swing_part_id]), (0, 255, 0), 2)
        if  correct_midpoint==0:
            cv2.line(frame, (data['midpoint_x'].iloc[swing_part_id], data['midpoint_y'].iloc[swing_part_id]), (data['midpoint_x'].iloc[swing_part_id], data['midpoint_y'].iloc[swing_part_id]-150),(0, 0, 255), 2)
        else:
            cv2.line(frame, (data['midpoint_x'].iloc[swing_part_id], data['midpoint_y'].iloc[swing_part_id]), (data['midpoint_x'].iloc[swing_part_id], data['midpoint_y'].iloc[swing_part_id]-150), (0, 255, 0), 2)

        if correct_arm_angle==0:
            cv2.putText(frame, f'Arm Angle: {data["arm_angle"].iloc[swing_part_id]:.2f}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(0, 0, 255), 2)
            cv2.line(frame, (data['left_shoulder_x'].iloc[swing_part_id], data['left_shoulder_y'].iloc[swing_part_id]), (data['left_elbow_x'].iloc[swing_part_id], data['left_elbow_y'].iloc[swing_part_id]), (0, 0, 255), 2)
            cv2.line(frame, (data['left_elbow_x'].iloc[swing_part_id], data['left_elbow_y'].iloc[swing_part_id]), (data['left_wrist_x'].iloc[swing_part_id], data['left_wrist_y'].iloc[swing_part_id]), (0, 0, 255), 2)
        else:
            cv2.putText(frame, f'Arm Angle: {data["arm_angle"].iloc[swing_part_id]:.2f}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(0, 255, 0), 2)
            cv2.line(frame, (data['left_shoulder_x'].iloc[swing_part_id], data['left_shoulder_y'].iloc[swing_part_id]), (data['left_elbow_x'].iloc[swing_part_id], data['left_elbow_y'].iloc[swing_part_id]), (0, 255, 0), 2)
            cv2.line(frame, (data['left_elbow_x'].iloc[swing_part_id], data['left_elbow_y'].iloc[swing_part_id]), (data['left_wrist_x'].iloc[swing_part_id], data['left_wrist_y'].iloc[swing_part_id]), (0, 255, 0), 2)
        
    elif swing_part=='contact':
        correct_head,correct_shoulder_ankle,correct_knee_angle,correct_arm_angle=evaluate_correctness(data, swing_part,swing_part_id,address_frame)
        # # Display angle and lines on the image
        cv2.putText(frame, f'Shoulders inclination: {data["shoulders_inclination"].iloc[swing_part_id]:.2f}', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        cv2.putText(frame, f'Hips inclination: {data["hips_inclination"].iloc[swing_part_id]:.2f}', (10, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

        if correct_knee_angle==1:
            cv2.putText(frame, f'Knee Angle: {data["knee_angle"].iloc[swing_part_id]:.2f}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            # cv2.line(frame, (data['left_hip_x'].iloc[swing_part_id], data['left_hip_y'].iloc[swing_part_id]), (data['left_knee_x'].iloc[swing_part_id], data['left_knee_x'].iloc[swing_part_id]), (0, 255, 0), 2)
            # cv2.line(frame, (data['left_knee_x'].iloc[swing_part_id], data['left_knee_y']).iloc[swing_part_id], (data['left_ankle_x'].iloc[swing_part_id], data['left_ankle_y'].iloc[swing_part_id]), (0, 255, 0), 2)
        else:
            cv2.putText(frame, f'Knee Angle: {data["knee_angle"].iloc[swing_part_id]:.2f}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            # cv2.line(frame, (data['left_hip_x'].iloc[swing_part_id], data['left_hip_y'].iloc[swing_part_id]), (data['left_knee_x'].iloc[swing_part_id], data['left_knee_x'].iloc[swing_part_id]), (0, 0, 255), 2)
            # cv2.line(frame, (data['left_knee_x'].iloc[swing_part_id], data['left_knee_y']).iloc[swing_part_id], (data['left_ankle_x'].iloc[swing_part_id], data['left_ankle_y'].iloc[swing_part_id]), (0, 0, 255), 2)

        if correct_shoulder_ankle==0:
            cv2.circle(frame, (data['left_ankle_x'].iloc[swing_part_id], data['left_ankle_y'].iloc[swing_part_id]), 6, (0, 255, 0), -1)
            cv2.line(frame, (data['left_ankle_x'].iloc[swing_part_id], data['left_ankle_y'].iloc[swing_part_id]), (data['left_ankle_x'].iloc[swing_part_id], data['left_ankle_y'].iloc[swing_part_id] - 200), (0, 255, 0), 2)
        else:
            cv2.circle(frame, (data['left_ankle_x'].iloc[swing_part_id], data['left_ankle_y'].iloc[swing_part_id]), 6, (0, 0, 255), -1)
            cv2.line(frame, (data['left_ankle_x'].iloc[swing_part_id], data['left_ankle_y'].iloc[swing_part_id]), (data['left_ankle_x'].iloc[swing_part_id], data['left_ankle_y'].iloc[swing_part_id] - 200),  (0, 0, 255), 2)
        

        if correct_head==0:
            cv2.line(frame, (data['nose_x'].iloc[address_frame.item()], data['nose_y'].iloc[address_frame.item()]), (data['nose_x'].iloc[swing_part_id], data['nose_y'].iloc[swing_part_id]), (0, 0, 255), 2)
            cv2.circle(frame, (data['nose_x'].iloc[swing_part_id], data['nose_y'].iloc[swing_part_id]), 6, (0,0,255), -1)
            cv2.circle(frame, (data['nose_x'].iloc[address_frame.item()], data['nose_y'].iloc[address_frame.item()]), 20, (0, 0, 255), 2)
            cv2.circle(frame, (data['nose_x'].iloc[address_frame.item()], data['nose_y'].iloc[address_frame.item()]), 6, (0, 0, 255), -1)
        else:
            cv2.line(frame, (data['nose_x'].iloc[address_frame.item()], data['nose_y'].iloc[address_frame.item()]), (data['nose_x'].iloc[swing_part_id], data['nose_y'].iloc[swing_part_id]), (0, 255, 0), 2)
            cv2.circle(frame, (data['nose_x'].iloc[swing_part_id], data['nose_y'].iloc[swing_part_id]), 6, (0, 255, 0), -1)
            cv2.circle(frame, (data['nose_x'].iloc[address_frame.item()], data['nose_y'].iloc[address_frame.item()]), 20, (0, 255, 0), 2)
            cv2.circle(frame, (data['nose_x'].iloc[address_frame.item()], data['nose_y'].iloc[address_frame.item()]), 6, (0, 255, 0), -1)


        if correct_arm_angle==0:
            cv2.putText(frame, f'Arm Angle: {data["arm_angle"].iloc[swing_part_id]:.2f}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(0, 0, 255), 2)
            cv2.line(frame, (data['left_shoulder_x'].iloc[swing_part_id], data['left_shoulder_y'].iloc[swing_part_id]), (data['left_elbow_x'].iloc[swing_part_id], data['left_elbow_y'].iloc[swing_part_id]), (0, 0, 255), 2)
            cv2.line(frame, (data['left_elbow_x'].iloc[swing_part_id], data['left_elbow_y'].iloc[swing_part_id]), (data['left_wrist_x'].iloc[swing_part_id], data['left_wrist_y'].iloc[swing_part_id]), (0, 0, 255), 2)
        else:
            cv2.putText(frame, f'Arm Angle: {data["arm_angle"].iloc[swing_part_id]:.2f}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(0, 255, 0), 2)
            cv2.line(frame, (data['left_shoulder_x'].iloc[swing_part_id], data['left_shoulder_y'].iloc[swing_part_id]), (data['left_elbow_x'].iloc[swing_part_id], data['left_elbow_y'].iloc[swing_part_id]), (0, 255, 0), 2)
            cv2.line(frame, (data['left_elbow_x'].iloc[swing_part_id], data['left_elbow_y'].iloc[swing_part_id]), (data['left_wrist_x'].iloc[swing_part_id], data['left_wrist_y'].iloc[swing_part_id]), (0, 255, 0), 2)

        
    elif swing_part=='top':
        correct_pelvis,correct_head,correct_arm_angle=evaluate_correctness(data, swing_part,swing_part_id,address_frame)

        cv2.putText(frame, f'Shoulders inclination: {data["shoulders_inclination"].iloc[swing_part_id]:.2f}', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.line(frame, (data['right_shoulder_x'].iloc[swing_part_id], data['right_shoulder_y'].iloc[swing_part_id]), (data['right_shoulder_x'].iloc[swing_part_id] + 100, data['right_shoulder_y'].iloc[swing_part_id]), (0, 255, 0), 2)
        cv2.line(frame, (data['left_shoulder_x'].iloc[swing_part_id], data['left_shoulder_y'].iloc[swing_part_id]), (data['right_shoulder_x'].iloc[swing_part_id], data['right_shoulder_y'].iloc[swing_part_id]), (0, 255, 0), 2)

        cv2.putText(frame, f'Hips inclination: {data["hips_inclination"].iloc[swing_part_id]:.2f}', (10, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

        # cv2.putText(frame, f'Knee Angle: {data["knee_angle"].iloc[swing_part_id]:.2f}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)

        if correct_pelvis==0:
            cv2.putText(frame, f'Pelvis Angle: {data["pelvis_angle"].iloc[swing_part_id]:.2f}', (10, 95), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            cv2.putText(frame, f'Pelvis Angle: {data["pelvis_angle"].iloc[swing_part_id]:.2f}', (10, 95), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


        if correct_head==0:
            cv2.line(frame, (data['nose_x'].iloc[address_frame.item()], data['nose_y'].iloc[address_frame.item()]), (data['nose_x'].iloc[swing_part_id], data['nose_y'].iloc[swing_part_id]), (0, 0, 255), 2)
            cv2.circle(frame, (data['nose_x'].iloc[swing_part_id], data['nose_y'].iloc[swing_part_id]), 6, (0,0,255), -1)
            cv2.circle(frame, (data['nose_x'].iloc[address_frame.item()], data['nose_y'].iloc[address_frame.item()]), 20, (0, 0, 255), 2)
            cv2.circle(frame, (data['nose_x'].iloc[address_frame.item()], data['nose_y'].iloc[address_frame.item()]), 6, (0, 0, 255), -1)
        else:
            cv2.line(frame, (data['nose_x'].iloc[address_frame.item()], data['nose_y'].iloc[address_frame.item()]), (data['nose_x'].iloc[swing_part_id], data['nose_y'].iloc[swing_part_id]), (0, 255, 0), 2)
            cv2.circle(frame, (data['nose_x'].iloc[swing_part_id], data['nose_y'].iloc[swing_part_id]), 6, (0, 255, 0), -1)
            cv2.circle(frame, (data['nose_x'].iloc[address_frame.item()], data['nose_y'].iloc[address_frame.item()]), 20, (0, 255, 0), 2)
            cv2.circle(frame, (data['nose_x'].iloc[address_frame.item()], data['nose_y'].iloc[address_frame.item()]), 6, (0, 255, 0), -1)

        if correct_arm_angle==0:
            cv2.putText(frame, f'Arm Angle: {data["arm_angle"].iloc[swing_part_id]:.2f}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(0, 0, 255), 2)
            cv2.line(frame, (data['left_shoulder_x'].iloc[swing_part_id], data['left_shoulder_y'].iloc[swing_part_id]), (data['left_elbow_x'].iloc[swing_part_id], data['left_elbow_y'].iloc[swing_part_id]), (0, 0, 255), 2)
            cv2.line(frame, (data['left_elbow_x'].iloc[swing_part_id], data['left_elbow_y'].iloc[swing_part_id]), (data['left_wrist_x'].iloc[swing_part_id], data['left_wrist_y'].iloc[swing_part_id]), (0, 0, 255), 2)
        else:
            cv2.putText(frame, f'Arm Angle: {data["arm_angle"].iloc[swing_part_id]:.2f}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(0, 255, 0), 2)
            cv2.line(frame, (data['left_shoulder_x'].iloc[swing_part_id], data['left_shoulder_y'].iloc[swing_part_id]), (data['left_elbow_x'].iloc[swing_part_id], data['left_elbow_y'].iloc[swing_part_id]), (0, 255, 0), 2)
            cv2.line(frame, (data['left_elbow_x'].iloc[swing_part_id], data['left_elbow_y'].iloc[swing_part_id]), (data['left_wrist_x'].iloc[swing_part_id], data['left_wrist_y'].iloc[swing_part_id]), (0, 255, 0), 2)


        # cv2.line(frame, (data['left_ankle_x'].iloc[swing_part_id], data['left_ankle_y'].iloc[swing_part_id]), (data['left_ankle_x'].iloc[swing_part_id], data['left_ankle_y'].iloc[swing_part_id] - 200), (255, 0, 0), 2)

    return frame
        

In [71]:
def split_swing(data):
    data=data.reset_index()
    data.columns = [col.replace(' X', '_x').replace(' Y', '_y').lower() for col in data.columns]
    
    #consider only time between backswing and finish
    halfway_back_ind=data['right_wrist_x'].idxmin()
    halfway_front_ind=data.right_wrist_x[data.index>halfway_back_ind].idxmax()
    middle_data=data[(data.index>halfway_back_ind)&(data.index<halfway_front_ind)]
    if middle_data.empty:
        halfway_back_ind=data.right_wrist_x[data.index<halfway_back_ind].idxmin()
        halfway_front_ind=data.right_wrist_x[data.index>halfway_back_ind].idxmax()
        middle_data=data[(data.index>halfway_back_ind)&(data.index<halfway_front_ind)]     
    #find moment of ball contact as the lowest wrist point on y
    contact_frame=middle_data[middle_data['right_wrist_y']==middle_data['right_wrist_y'].max()].index
    if len(contact_frame):
        contact_frame=contact_frame[0]
    #isolate only backswing data
    back_data=data[(data.index<contact_frame.item())]
    #find moment of top of backswing as the highest wrist point on y
    top_backswing_frame=back_data[back_data['right_wrist_y']==back_data['right_wrist_y'].min()].index
    if len(top_backswing_frame)>1:
        top_backswing_frame=top_backswing_frame[0]

    #find moment before start of the swing as the lowest wrist point on y before going halfway back
    halfway_back_data=data[data.index<halfway_back_ind]
    address_frame=halfway_back_data[halfway_back_data['right_wrist_y']==halfway_back_data['right_wrist_y'].max()].index
    if len(address_frame)>1:
        address_frame=address_frame[0]

    return data,address_frame,contact_frame,top_backswing_frame

In [72]:

# name_list=list('output_video_'+pd.unique(test_dat.youtube_id))
name_list= [file.split('.')[0] for file in os.listdir(directory_path) if file.endswith('.csv')]
all=len(name_list)
correct_contact=0
correct_top=0
correct_address=0
swing_part='top'
folder_name = 'frames_mediapipe_'+swing_part
full_path = os.path.join(directory_path, folder_name)

if not os.path.exists(full_path):
    os.makedirs(full_path)


for fl in name_list:
    # try:
        video_files = [file for file in os.listdir(original_videos_path) if fl in file and file.endswith('.mp4')]
        if video_files==[]:
            print('file not included')
            continue
        video_path = original_videos_path + video_files[0]
        cap = cv2.VideoCapture(video_path)

        data = pd.read_csv(directory_path + fl + '.csv')
        data,address_frame,contact_frame,top_backswing_frame=split_swing(data)
        #extract the gt from golfDB
        yt_id='_'.join(fl.split('_')[2::])
        gt_contact=test_dat[test_dat.youtube_id==yt_id].events_7.item()
        gt_top=test_dat[test_dat.youtube_id==yt_id].events_5.item()
        gt_address=15

        #calculte tolerance as per golfDB paper (fps=30)
        tolerance_hit=2*(gt_contact//30)
        tolerance_top=2*(gt_top//30)
        tolerance_address=2*(gt_address//30)
        # break
        

        # Processing based on swing_part value
        if swing_part == 'address':
            swing_part_id=max([address_frame.item()-4,0])
            
            cap.set(cv2.CAP_PROP_POS_FRAMES, swing_part_id)
            ret, frame = cap.read()
            frame=plot_points(data,frame,'address',swing_part_id)
            frame_path =full_path+'/'+os.path.splitext(video_path)[0].split("/")[1] + f'_frame_{address_frame.item()}.jpg'
            cv2.imwrite(frame_path, frame)
            cap.release()
            
            if gt_address - tolerance_address <= address_frame.item() <= tolerance_address + gt_address:
                correct_address += 1
        
        
        elif swing_part == 'contact':
            swing_part_id=contact_frame.item()
            cap.set(cv2.CAP_PROP_POS_FRAMES, swing_part_id)
            ret, frame = cap.read()

            frame=plot_points(data,frame,swing_part,swing_part_id,address_frame)
            frame_path = full_path+ '/'+os.path.splitext(video_path)[0].split("/")[1] + f'_frame_{contact_frame.item()}.jpg'
            cv2.imwrite(frame_path, frame)

            cap.release()

            if gt_contact - tolerance_hit <= contact_frame.item() <= tolerance_hit + gt_contact:
                correct_contact += 1

        elif swing_part == 'top':
            swing_part_id=top_backswing_frame.item()
            cap.set(cv2.CAP_PROP_POS_FRAMES, swing_part_id)
            ret, frame = cap.read()
            frame=plot_points(data,frame,swing_part,swing_part_id,address_frame)
            frame_path =full_path+ '/'+os.path.splitext(video_path)[0].split("/")[1] + f'_frame_{top_backswing_frame.item()}.jpg'
            cv2.imwrite(frame_path, frame)
            cap.release()
            if gt_top-tolerance_top<=top_backswing_frame.item()<=tolerance_top+gt_top:
                correct_top+=1

        
    # except:
    #     continue
if swing_part == 'address':
    accuracy_address=correct_address/all
    print(accuracy_address)
elif swing_part == 'contact':
    accuracy_contact=correct_contact/all
    print(accuracy_contact)
elif swing_part == 'top':
    accuracy_top=correct_top/all
    print(accuracy_top)


0.6127167630057804


In [73]:
a=np.array([[2],[2]])
a.shape

(2, 1)